In [1]:
import pandas as pd
import os


file_paths = {
    "BDI": "/content/BDI monthly average.xlsx",
    "CPI": "/content/CPI Growth Rate (Seasonally Adjusted).xlsx",
    "CRB": "/content/CRB Spot Index monthly average.xlsx",  # Target variable
    "EFFR": "/content/Effective Federal Funds Rate (Month Average).xlsx",
    "Fed_Assets": "/content/Fed Reserve Total Asset.xlsx",
    "IPI": "/content/IPI Growth Rate (Seasonally Adjusted).xlsx",
    "PCE": "/content/PCE Price Index.xlsx",
    "SP500": "/content/S&P 500 Index (SPX).xlsx",
    "Unemployment": "/content/Unemployment Rate (Seasonally Adjusted).xlsx",
    "USD_Index": "/content/US Dollar Index (Spot Public).xlsx"
}


dfs = {}
for name, path in file_paths.items():
    try:
        dfs[name] = pd.read_excel(path)
    except Exception as e:
        print(f"Ошибка при чтении {name}: {e}")


{key: df.head() for key, df in dfs.items()}


{'BDI':            Unnamed: 0                              Baltic Dry Index
 0        Data Sources  Baltic Exchange Information Services Limited
 1                Mean                                   1870.605825
 2            Variance                                2473515.921522
 3  Standard Deviation                                    1572.74153
 4            Skewness                                       3.04228,
 'CPI':            Unnamed: 0  CPI Growth rate Seasonally Adjusted
 0                 NaN  %Monthly Year-over-Year Growth Rate
 1                Base                   base 1982-1984=100
 2                Mean                             4.017563
 3            Variance                             8.476462
 4  Standard Deviation                             2.911436,
 'CRB':            Unnamed: 0      CRB Spot Index
 0                Base            1967=100
 1        Data Sources  Barchart.com, Inc.
 2                Mean           313.55343
 3            Variance        1

In [4]:

cleaned_dfs = {}

for key, df in dfs.items():
    date_col = df.columns[0]
    df_cleaned = df.iloc[5:].reset_index(drop=True)
    df_cleaned[date_col] = pd.to_datetime(df_cleaned[date_col], errors='coerce')
    df_cleaned = df_cleaned.dropna(subset=[date_col])
    df_cleaned = df_cleaned.rename(columns={df_cleaned.columns[1]: key})
    df_cleaned = df_cleaned[[date_col, key]]
    cleaned_dfs[key] = df_cleaned


merged_df = list(cleaned_dfs.values())[0]
for key, df in list(cleaned_dfs.items())[1:]:
    merged_df = pd.merge(merged_df, df, on=df.columns[0], how="outer")


merged_df = merged_df.rename(columns={merged_df.columns[0]: "Date"})


merged_df = merged_df.sort_values("Date").reset_index(drop=True)

merged_df


<ipython-input-4-8615f978eb85>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[date_col] = pd.to_datetime(df_cleaned[date_col], errors='coerce')
<ipython-input-4-8615f978eb85>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[date_col] = pd.to_datetime(df_cleaned[date_col], errors='coerce')
<ipython-input-4-8615f978eb85>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cleaned[date_col] = pd.to_datetime(df_cleaned[date_col], errors='coerce')
<ipython-input-4-8615f978eb85>:6: UserWarning: Could not infer format, so each element will be parsed individ

,Date,BDI,CPI,CRB,EFFR,Fed_Assets,IPI,PCE,SP500,Unemployment,USD_Index
0,1970-01-01,NaN,6.162465,116.075,8.959048,17.396,-0.675886,19.377,NaN,3.9,NaN
1,1970-02-01,NaN,6.424581,116.6,8.974211,17.67,-1.372579,19.454,NaN,4.2,NaN
2,1970-03-01,NaN,6.094183,116.94,7.710455,17.35,-2.266824,19.502,NaN,4.4,NaN
3,1970-04-01,NaN,6.060606,116.3,8.085909,16.919,-2.158066,19.601,NaN,4.6,NaN
4,1970-05-01,NaN,6.043956,114.975,7.859048,16.165,-1.901452,19.663,NaN,4.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...
656,2024-09-01,1964.809524,2.432541,534.4775,5.13,250.619,-0.697719,123.939,5762.48,4.1,101.099524
657,2024-10-01,1666.652174,2.571404,534.22381,4.83,243.925,-0.444734,124.235,5705.45,4.1,103.176087
658,2024-11-01,1539.666667,2.714168,534.761,4.645789,241.464,-0.887577,124.387,6032.38,4.2,105.767619
659,2024-12-01,1099.235294,2.872366,538.499048,4.484762,238.801,0.343366,124.705,5881.63,4.1,107.071905


In [6]:
missing_values = merged_df.isnull().sum()

high_missing_cols = missing_values[missing_values > 0.3 * len(merged_df)].index


merged_df_cleaned = merged_df.drop(columns=high_missing_cols)

merged_df_cleaned = merged_df_cleaned.fillna(method="pad")

merged_df_cleaned = merged_df_cleaned.fillna(method="bfill")


columns_to_diff = ["CRB", "Fed_Assets", "PCE", "SP500", "USD_Index"]

for col in columns_to_diff:
    if col in merged_df_cleaned.columns:
        merged_df_cleaned[col + "_Change"] = merged_df_cleaned[col].pct_change()
        merged_df_cleaned = merged_df_cleaned.drop(columns=[col])


merged_df_cleaned = merged_df_cleaned.dropna().reset_index(drop=True)

merged_df_cleaned

<ipython-input-6-9a05a9067a36>:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df_cleaned = merged_df_cleaned.fillna(method="pad")
<ipython-input-6-9a05a9067a36>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df_cleaned = merged_df_cleaned.fillna(method="pad")
<ipython-input-6-9a05a9067a36>:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df_cleaned = merged_df_cleaned.fillna(method="bfill")


,Date,BDI,CPI,EFFR,IPI,Unemployment,CRB_Change,Fed_Assets_Change,PCE_Change,SP500_Change,USD_Index_Change
0,1970-02-01,974.200000,6.424581,8.974211,-1.372579,4.2,0.004523,0.015751,0.003974,0.000000,0.000000
1,1970-03-01,974.200000,6.094183,7.710455,-2.266824,4.4,0.002916,-0.018110,0.002467,0.000000,0.000000
2,1970-04-01,974.200000,6.060606,8.085909,-2.158066,4.6,-0.005473,-0.024841,0.005076,0.000000,0.000000
3,1970-05-01,974.200000,6.043956,7.859048,-1.901452,4.8,-0.011393,-0.044565,0.003163,0.000000,0.000000
4,1970-06-01,974.200000,6.010929,7.540455,-3.164622,4.9,-0.010394,0.010084,0.002594,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
655,2024-09-01,1964.809524,2.432541,5.130000,-0.697719,4.1,0.001140,0.006122,0.001713,0.020197,-0.011968
656,2024-10-01,1666.652174,2.571404,4.830000,-0.444734,4.1,-0.000475,-0.026710,0.002388,-0.009897,0.020540
657,2024-11-01,1539.666667,2.714168,4.645789,-0.887577,4.2,0.001006,-0.010089,0.001223,0.057301,0.025118
658,2024-12-01,1099.235294,2.872366,4.484762,0.343366,4.1,0.006990,-0.011029,0.002557,-0.024990,0.012332


In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import boxcox

In [15]:
df = merged_df_cleaned
df = df.drop(columns=["Date"], errors="ignore")


correlation_matrix = df.corr().abs()
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
df = df.drop(columns=to_drop)


rolling_windows = [3, 6, 12]
for window in rolling_windows:
    df[f'CRB_MA_{window}'] = df['CRB_Change'].rolling(window=window).mean()


lags = [1, 2, 3, 6]
for lag in lags:
    df[f'CRB_Lag_{lag}'] = df['CRB_Change'].shift(lag)


Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


scaler = StandardScaler()
df.iloc[:, :] = scaler.fit_transform(df)


df = df.dropna()


processed_file_path = "/Enhanced_Processed_Financial_Data.csv"
df.to_csv(processed_file_path, index=False)



In [16]:
df

,BDI,CPI,EFFR,IPI,Unemployment,CRB_Change,Fed_Assets_Change,PCE_Change,SP500_Change,USD_Index_Change,CRB_MA_3,CRB_MA_6,CRB_MA_12,CRB_Lag_1,CRB_Lag_2,CRB_Lag_3,CRB_Lag_6
11,-0.648802,1.108049,-0.062507,-0.956229,-0.004867,-0.152712,0.882813,-0.131684,-0.265558,0.027316,-0.918684,-0.896629,-0.843779,-1.055162,-0.894768,-0.589855,-0.358654
12,-0.648802,0.792228,-0.269236,-0.989725,-0.004867,1.385111,-0.750163,0.311956,-0.265558,-0.058985,0.116886,-0.464857,-0.613835,-0.097531,-1.035133,-0.859477,0.042736
13,-0.648802,0.628917,-0.253729,-0.983991,0.059813,-0.358894,-0.875523,0.529622,-0.265558,0.002241,0.442620,-0.424937,-0.698483,1.464755,-0.084155,-0.996308,-0.344815
14,-0.648802,0.467303,-0.104589,-0.763925,-0.004867,0.120237,-0.195407,0.883781,-0.265558,0.052638,0.563346,-0.190655,-0.600691,-0.306993,1.467280,-0.069275,-0.524517
15,-0.648802,0.609202,0.055947,-0.594989,-0.004867,-0.575257,-2.230572,1.013135,-0.265558,-0.385716,-0.303727,-0.079409,-0.570954,0.179760,-0.292161,1.443095,-0.788745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,1.797593,-0.517534,0.209432,-0.845977,-1.169103,-0.015711,0.345312,-0.366396,0.340493,-0.685983,-0.468373,0.078040,-0.313705,-0.583404,-0.474342,-0.133511,1.053942
656,1.061268,-0.438177,0.111671,-0.777310,-1.169103,-0.107943,-1.728002,0.014441,-0.562532,1.251546,-0.277506,-0.396097,-0.186213,0.041649,-0.566652,-0.449637,1.593649
657,0.747666,-0.356589,0.051642,-0.897510,-1.104423,-0.023416,-0.678420,-0.642906,1.453893,1.524395,-0.008781,-0.330160,0.076251,-0.052050,0.054059,-0.539623,-0.107357
658,-0.340017,-0.266182,-0.000832,-0.563397,-1.169103,0.318336,-0.737742,0.109402,-1.015441,0.762316,0.138968,-0.173533,0.491784,0.033822,-0.038989,0.065459,-0.077302
